In [3]:
import json

In [4]:
with open(f'data/landing/fantasy/2022/driver_results.json', 'r') as f:
        f2022 = json.load(f)

In [5]:
with open(f'data/landing/fantasy/2023/driver_results.json', 'r') as f:
        f2023 = json.load(f)

In [6]:
with open(f'data/landing/fantasy/2023/driver_results.json', 'r') as f:
        f2024 = json.load(f)

In [8]:
len(f2024[0]['race_results'][0]['fantasy_results'])

16

In [9]:
len(f2023[0]['race_results'][0]['fantasy_results'])

16

In [10]:
len(f2022[0]['race_results'][0]['fantasy_results'])

16

In [13]:
for i in range(len(f2022[0]['race_results'][0]['fantasy_results'])):
    print(f2022[0]['race_results'][0]['fantasy_results'][i]['id'])

quali_not_classified_points
quali_disqualified_points
quali_pos_points
sprint_pos_gained_points
sprint_overtake_points
sprint_fastest_lap_points
sprint_not_classified_points
sprint_disqualified_points
sprint_pos_points
race_pos_gained_points
race_overtake_points
race_fastest_lap_points
race_not_classified_points
race_disqualified_points
race_pos_points
race_dotd_points


In [12]:
len(['quali_not_classified_points',
                        'quali_disqualified_points',
                        'quali_pos_points',
                        'quali_teamwork_points',
                        'sprint_pos_gained_points',
                        'sprint_overtake_points',
                        'sprint_fastest_lap_points',
                        'sprint_not_classified_points',
                        'sprint_disqualified_points',
                        'sprint_pos_points',
                        'race_pos_gained_points',
                        'race_overtake_points',
                        'race_fastest_lap_points',
                        'race_not_classified_points',
                        'race_disqualified_points',
                        'race_pos_points',
                        'race_pit_stop_points'])

17